In [1]:
# import the required libraries
import os
import re
import csv
import pandas as pd
import numpy as np
import pyodbc
import Levenshtein as lev


In [2]:
# To link the DRC with the boekzaallijst we decided to follow 6 strategies. 

# Strategies to link DRC and BZ are: 
# 1. the first letter of the name, the full surname and the year of the first time someone acted as minister.
# 2. the first letter of the name, the full surname and the year of the first time someone acted as minister -1, since there can be a delay in the boekzaallijst registration.
# 3. The first 3 of the surname and the year of the first time someone acted as minister.   
# 4. The first 3 of the surname and the year of the first time someone acted as minister -1 (see 2).
# 5. With the strings created in strategy 1 apply a top 3 matching based on Levenshtein distances

# Before we start we load the "boekzaalijst" data from a csv file.

In [3]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'boekzaallijst_27072023.csv')
folder_output = 'output//'
output_csv = folderlink+folder_output+'clerus_boekzaal.csv'
drc_database = 'DRC_05102023_merged.accdb'


# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Load the boekzaallijst dataset from a csv
years_to_integers = {'Jaar (Begin) Rol': pd.Int64Dtype(),'Jaar Beroepen': pd.Int64Dtype(), }
boekzaallijst = pd.read_csv(input_file, sep=';', dtype=years_to_integers, encoding='utf-8')

In [5]:
def get_first_letter(row, name_column, initial_column):
    name_letter = row[name_column][0] if pd.notnull(row[name_column]) else None
    initial_letter = row[initial_column][0] if pd.notnull(row[initial_column]) else None
    return name_letter or initial_letter

In [6]:
# Create the new field containing the 'First_Letter'
boekzaallijst['first_letter'] = boekzaallijst.apply(lambda row: get_first_letter(row, 'Voornaam_BZ', 'Voorletter_BZ'), axis=1)

In [7]:
fil_boekzaallijst = boekzaallijst.dropna(subset=['Jaar Beroepen'])

In [8]:
# Create the link to formulate the connection using strategy 1
fil_boekzaallijst['strat1_boekzaallink'] = fil_boekzaallijst['first_letter'].astype(str) + '_' + fil_boekzaallijst['Achternaam_BZ'].astype(str) + '_' + fil_boekzaallijst['Jaar Beroepen'].astype(str).str.replace(' ', '')

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\848547381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_boekzaallijst['strat1_boekzaallink'] = fil_boekzaallijst['first_letter'].astype(str) + '_' + fil_boekzaallijst['Achternaam_BZ'].astype(str) + '_' + fil_boekzaallijst['Jaar Beroepen'].astype(str).str.replace(' ', '')


In [9]:
# Create the link to formulate the connection using strategy 2
def lower_one_to_integer(num):
    return num - 1

In [10]:
fil_boekzaallijst['year_min1'] = fil_boekzaallijst['Jaar Beroepen'].apply(lower_one_to_integer)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\4270696410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_boekzaallijst['year_min1'] = fil_boekzaallijst['Jaar Beroepen'].apply(lower_one_to_integer)


In [11]:
fil_boekzaallijst['strat2_boekzaallink'] = fil_boekzaallijst['first_letter'].astype(str) + '_' + fil_boekzaallijst['Achternaam_BZ'].astype(str) + '_' + fil_boekzaallijst['year_min1'].astype(str).str.replace(' ', '')

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\2022391221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_boekzaallijst['strat2_boekzaallink'] = fil_boekzaallijst['first_letter'].astype(str) + '_' + fil_boekzaallijst['Achternaam_BZ'].astype(str) + '_' + fil_boekzaallijst['year_min1'].astype(str).str.replace(' ', '')


In [12]:
# Strategy 3 and 4 
fil_boekzaallijst['strat3_boekzaallink'] =  fil_boekzaallijst['Achternaam_BZ'].str[:3]+ '_' + fil_boekzaallijst['Jaar Beroepen'].astype(str).str.replace(' ', '')
fil_boekzaallijst['strat4_boekzaallink'] =  fil_boekzaallijst['Achternaam_BZ'].str[:3]+ '_' + fil_boekzaallijst['year_min1'].astype(str).str.replace(' ', '')


C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\2378997072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_boekzaallijst['strat3_boekzaallink'] =  fil_boekzaallijst['Achternaam_BZ'].str[:3]+ '_' + fil_boekzaallijst['Jaar Beroepen'].astype(str).str.replace(' ', '')
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\2378997072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_boekzaallijst['strat4_boekzaallink'] =  fil_boekzaallijst['Achternaam_BZ'].str[:3]+ '_' + fil_boek

In [14]:
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ='+folderlink+input_folder+drc_database+';'
)

In [15]:
# Establish the connection
conn = pyodbc.connect(conn_str)

# Read the table into a pandas DataFrame
# Replace 'your_table_name' with the name of the table you want to read.
drc_bio = pd.read_sql('SELECT * FROM 01_DRC_BIO', conn)
drc_role = pd.read_sql('SELECT * FROM 12_DRC_roles', conn)

# Close the connection
conn.close()

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\1759605146.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  drc_bio = pd.read_sql('SELECT * FROM 01_DRC_BIO', conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\1759605146.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  drc_role = pd.read_sql('SELECT * FROM 12_DRC_roles', conn)


In [16]:
def double_to_integer(dataframe, field):
    dataframe[field] = dataframe[field].astype('Int64')  


In [17]:
double_to_integer(drc_role, 'role_start_year')
double_to_integer(drc_role, 'role_end_year')
double_to_integer(drc_bio, 'birth_year')
double_to_integer(drc_bio, 'death_year')
double_to_integer(drc_bio, 'baptized_year')
double_to_integer(drc_bio, 'burried_year')

In [18]:
drc_joined = pd.merge(drc_bio, drc_role, left_on='drc_id', right_on='drc_id', how = 'right')


In [19]:
drc_subset = drc_joined[drc_joined['role_type'] == 'predikant']

In [20]:
drc_subset.dropna(subset=['role_start_year'], inplace=True)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\3354532571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drc_subset.dropna(subset=['role_start_year'], inplace=True)


In [21]:
first_minister_subset = drc_subset.loc[drc_subset.groupby('drc_id')['role_start_year'].idxmin()]

In [24]:
# Creating the linking field for strategy 1 and strategy 2

first_minister_subset['strat12_drc_link'] = first_minister_subset['first_letter'].astype(str) + '_' +first_minister_subset['surname'].astype(str) + '_' + first_minister_subset['role_start_year'].astype(str).str.replace(' ', '')

In [25]:
# Creating the linking field for strategy 3 and strategy 4
first_minister_subset['strat34_drc_link'] = first_minister_subset['surname'].str[:3] + '_' + first_minister_subset['role_start_year'].astype(str).str.replace(' ', '')


In [42]:
light_drc = first_minister_subset[['drc_id','original_input','role_place','strat34_drc_link','strat12_drc_link']]


In [43]:
light_bz = fil_boekzaallijst[['Nr_BZ','strat1_boekzaallink','strat2_boekzaallink','strat3_boekzaallink','strat4_boekzaallink']]

In [45]:
strategy1 = pd.merge(light_bz, light_drc, left_on='strat1_boekzaallink', right_on='strat12_drc_link', how='inner')
strategy1['strategy'] = 1
strategy2 = pd.merge(light_bz, light_drc, left_on='strat2_boekzaallink', right_on='strat12_drc_link', how='inner')
strategy2['strategy'] = 2
strategy3 = pd.merge(light_bz, light_drc, left_on='strat3_boekzaallink', right_on='strat34_drc_link', how='inner')
strategy3['strategy'] = 3
strategy4 = pd.merge(light_bz, light_drc, left_on='strat4_boekzaallink', right_on='strat34_drc_link', how='inner')
strategy4['strategy'] = 4

In [71]:
appended_strategies = pd.concat([strategy1, strategy2, strategy3, strategy4], ignore_index=True)


In [78]:
# Compute a cross join
light_bz['key'] = 1
light_drc['key'] = 1
cross_df = light_bz.merge(light_drc, on='key').drop('key', axis=1)
string1 = 'strat1_boekzaallink'
string2 = 'strat12_drc_link'
cross_df['lev_ratio'] = cross_df.apply(lambda row: lev.ratio(row[string1], row[string2]), axis=1)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\3115096322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  light_bz['key'] = 1
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_8432\3115096322.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  light_drc['key'] = 1


In [79]:
cross_df = cross_df.sort_values(by='lev_ratio', ascending=False)

In [80]:
def top_n_matches(group, n=3):
    return group.nlargest(n, 'lev_ratio')

In [96]:
strategy5 = cross_df.groupby('Nr_BZ').apply(top_n_matches).reset_index(drop=True)

In [106]:
strategy5['strategy'] = 5

In [107]:
appended_strategies_lev = pd.concat([appended_strategies, strategy5], ignore_index=True)


In [109]:
appended_strategies_lev.to_csv(folderlink+folder_output+'possible_links_drc_bz_strat1-5.csv', sep=';', encoding='utf-8', index=False)